# Alergia Algorithm applied to Trip Advisor data.

## Importation des packages nécessaires

In [1227]:
import pandas as pd
import datetime
import warnings
import numpy as np
import matplotlib.pyplot as plt
import math
import itertools

## Fonction d'ouverture du csv.

On drop un certain nombre de colonnes (souvent données géographiques car pas nécessaires pour l'analyse). On drop aussi les lignes ou des données sont manquantes (notamment quand la nationalité n'est pas renseignée)

In [1228]:
def ouverture(nom):
    df = pd.read_csv(nom, sep = "," )
    df = df.drop(columns = ['Unnamed: 0', 'gid_0', 'name_0', 'gid_1','name_1','gid_2', 'name_2', 'gid_3' ,'name_3', 'gid_4', 'name_4', 'gid_5', 'name_5', 'idplace'])
    df['date_review'] = df['date_review'].replace('0000-00-00', np.nan)
    df = df.dropna()
    df['date_review'] = pd.to_datetime(df['date_review'], format = '%Y-%m-%d')
    df = df[df.country != '-']
    return df

In [1229]:
df = ouverture('sample_data.csv')
df.head()

,nom,latitude,longitude,id,idauteur,date_review,country
0,"L'Insoumise, France","48,835045","2,28915",328246480,AEC731BAB7B8CCA02944C293198E3FC6,2015-11-21,Italy
1,Cinq Mondes Paris,"48,871681","2,32899",331590265,26ABC72A68434467E4D20CEC4F1B3DC3,2015-12-09,France
2,"L'Envie, France","48,83414","2,317883",262837346,36B0E60FD21BFE2377C526209C3FF4D5,2015-03-31,France
3,"Le Ciel de Paris, France","48,84219","2,32198",266833774,72C94635808B0207A5A664DD646F3744,2015-04-20,France
4,"Cafe Les Deux Magots, France","48,853973","2,333158",304112023,F007B2D8152512B20BB4EB3E7C5ACA2C,2015-08-27,Brazil


Nombre de lieux différents

In [1230]:
len(list(set(df['nom'])))

9420

Nombre de reviews totales

In [1231]:
len(df)

56264

## Selection de l'année pour l'analyse

In [1232]:
def selection_annee(annee, df):
    df = df[(df['date_review'].dt.year > annee - 1) & (df['date_review'].dt.year < annee + 1)]
    return df

In [1233]:
df2 = selection_annee(2015, df)

Nombre de reviews en 2015

In [1234]:
len(df2)

56264

## Selection du nombre de lieux à traiter (les top k premiers)

Pour se faire, on groupby le nom du lieu, auquel on applique la fonction count du nombre unique de review (représenté par la colonne id). On trie ensuite le résultat pour avoir le top de lieux représentés

In [1235]:
def top_k(k, df):
    top_k = df.groupby('nom').count()['id'].sort_values(ascending = False)[0:k]
    return top_k

In [1293]:
tp_k = top_k(10, df2)

In [1294]:
tp_k

nom
Musée du Louvre                          1549
Eiffel Tower                             1516
Cathédrale Notre-Dame de Paris            749
Arc de Triomphe                           711
Jardin du Luxembourg                      589
La Seine                                  478
Musée d'Orsay                             413
Basilique du Sacré-Cœur de Montmartre     367
Champs-Élysées                            348
Opéra Garnier                             333
Name: id, dtype: int64

## On peut maintenant réduire le DataFrame au top k de lieux 

In [1295]:
def to_keep(df, top_k):
    keep = pd.DataFrame(columns = df.columns)
    for places in top_k.index:
        keep = keep.append(df[df['nom'] == places])
    return keep

In [1296]:
df3 = to_keep(df2, tp_k).reset_index()

On a le pourcentage que représente le top_K par rapport à la totalité des reviews de l'année choisie

In [1297]:
len(df3.index)/len(df2.index)

0.12535546708374803

## Afin de créer les séquences, par soucis de clarté et de traitement, on va assigner un numéro à chaque lieux. Puis, le réduire à 1 digit via la colonne ascii.

On pourra modifier plus tard l'algorithme Alergia afin qu'il puisse prendre en compte les doubles digits

In [1298]:
df3['Group_ID'] = df3.groupby('nom').grouper.group_info[0]
df3['ascii'] = df3.Group_ID.apply(lambda x: chr(x+48))
df3.head()

,index,nom,latitude,longitude,id,idauteur,date_review,country,Group_ID,ascii
0,8,Musée du Louvre,"48,861","2,335833",311501513,4B018C46529B92190134DEBD3D429BCD,2015-09-18,Italy,8,8
1,76,Musée du Louvre,"48,861","2,335833",288015394,294FB3D2C00277C2A2F563F2D4DA7A55,2015-07-11,Ireland,8,8
2,117,Musée du Louvre,"48,861","2,335833",316627370,39AE30708C84823D0A6DEFB86D7D2370,2015-10-06,Argentina,8,8
3,143,Musée du Louvre,"48,861","2,335833",277869091,2D508690DD4E5281BF88B235E1121A2C,2015-06-04,Turkey,8,8
4,152,Musée du Louvre,"48,861","2,335833",249606817,F6883072C58471A6A04D7FBFB9608DA2,2015-01-16,Brazil,8,8


In [1299]:
len(list(set(df3.idauteur)))

6424

In [1300]:
def asciiToName(df):
    testdata = df[['ascii', 'nom']].values
    return dict(list(set(tuple(x) for x in testdata)))

In [1301]:
convert = asciiToName(df3)

On peut ensuite avoir la correspondance des digit des séquences aux lieux 

In [1302]:
convert

{'7': "Musée d'Orsay",
 '5': 'Jardin du Luxembourg',
 '0': 'Arc de Triomphe',
 '2': 'Cathédrale Notre-Dame de Paris',
 '3': 'Champs-Élysées',
 '9': 'Opéra Garnier',
 '4': 'Eiffel Tower',
 '6': 'La Seine',
 '8': 'Musée du Louvre',
 '1': 'Basilique du Sacré-Cœur de Montmartre'}

## On passe maintenant à la création des séquences pour chaque utilisateur. La particularité est qu'on va séparer les séquences en fonction du nombre de jours entre 2 photos.

On commence par groupby les lieux dans l'ordre chronologique en fonction de l'id de l'auteur. Autrement dit, on va mettre dans une liste les lieux reviewés avec leur date de publication respective de chaque utilisateur. 
Puis, on va comparer le delta de temps entre la review i et la i + 1. Si c'est inférieur ou égal au paramètre qu'on souhaite, on ajoute l'élément à la séquence *a* . Sinon, on termine la séquence, et on sépare les deux reviews en 2 séquences *a* et *b*  distinctes.

In [1303]:
def create_sequencesv2(df, threshold_days):
    df2 = df.sort_values('date_review', ascending = True).groupby('idauteur').apply(lambda x: x[['ascii','date_review']].values.tolist()).reset_index(name='col')
    
    df2['len'] = df2['col'].map(len)
    df2 = df2[df2['len'] >= 2] 

    sequences_finales = [] #used to split patterns according to the threshold of days we want between 2 days
            
    for element in df2['col']:
        sublist = ''
        sublist2 = ''
        for i in range(1, len(element)):
            
            diff = abs(element[i-1][1] - element[i][1]).days
            
            if(diff <= threshold_days): #IF THE PICTURE AND THE PREVIOUS ONE HAVE LESS THAN X DAYS 
                
                if(i == 1): #if at the beginning of the sequence
                    sublist += element[i-1][0] #we add the first element of the sequence
        
                sublist += element[i][0] #if not, we add the current one to the sublist
                   
            else: #IF THERE IS MORE THAN X DAYS :
                
                if(len(sublist) != 0): #if we stop a existing sequence, we add it
                    sequences_finales.append(sublist)  
                    sublist = ''
                
                if(i == 1): #if at the beginning of the sequence
                    sublist += element[i-1][0] #we add the first one
                    sequences_finales.append(sublist)  #then we cut the sub-sequence
                    sublist ='' #and we reset the sub-sequence
                    
                sublist += element[i][0] #if not, we create another sub-sequence
                
                    
        if(len(sublist) != 0): #at the end, we add the subsequence
            sequences_finales.append(sublist)
                    
    lieux = df['nom'].unique() 
    
    return lieux, sequences_finales

In [1304]:
lieux, sequences = create_sequencesv2(df3, 7) #utiliser 7 pour être en accord avec l'état de l'art

on garde que les séquences de taille > 2

In [1305]:
def cutSeq(sequences):
    seq2j = []
    for s in sequences:
        if len(s) >= 2:
            seq2j.append(s)
    return seq2j

In [1306]:
seq2j = cutSeq(sequences)

In [1307]:
def averageLen(lst):
    lengths = [len(i) for i in lst]
    return 0 if len(lengths) == 0 else (float(sum(lengths)) / len(lengths)) 

taille moyenne des séquences de touristes

In [1308]:
averageLen(seq2j) 

2.0862533692722374

nombre totales de séquences

In [1309]:
len(seq2j)

371

In [1310]:
maxx = 0
for s in seq2j:
    if(len(s) > maxx):
        maxx = len(s)
print(maxx)

4


Voici les 5 premières séquences que nous return l'algorithm. Nous n'avons accès à aucune donnée utilisateur, seulement à des séquence d'items (chaque symbole représente un lieu).

In [1311]:
seq2j[:5]

['96', '10', '72', '19', '806']

### Pour les séquences, ça part de 0 jusqu'a len. 
#### Tous les paramètres sont donc la position souhaitée - 1 (0 pour 1er élement, 1 pour 2ème, ...)

In [1312]:
def getSeqAtPosition(sequences, element, position):
    result = []
    for seq in sequences:
        for i in range(len(seq)):
            if(i == position):
                if(seq[i] == element):
                    result.append(seq)
    return result

In [1313]:
tourEiffel1 = getSeqAtPosition(seq2j[:1000], str(2), 0)

In [1314]:
tourEiffel2 = getSeqAtPosition(seq2j[:1000], str(2), 1)

In [1315]:
tourEiffel3 = getSeqAtPosition(seq2j[:1000], str(2), 2)

In [1316]:
notreDame1 = getSeqAtPosition(seq2j[:1000], str(1), 0)

In [1317]:
notreDame2 = getSeqAtPosition(seq2j[:1000], str(1), 1)

In [1318]:
notreDame3 = getSeqAtPosition(seq2j[:1000], str(1), 2)

In [1319]:
laSeine1 = getSeqAtPosition(seq2j[:1000], str(4), 0)

In [1320]:
laSeine2 = getSeqAtPosition(seq2j[:1000], str(4), 1)

In [1321]:
laSeine3 = getSeqAtPosition(seq2j[:1000], str(4), 2)

In [1322]:
louvre1 = getSeqAtPosition(seq2j[:1000], str(5), 0)

In [1355]:
louvre2 = getSeqAtPosition(seq2j[:1000], str(5), 1)

In [1356]:
louvre3 = getSeqAtPosition(seq2j[:1000], str(5), 2)

In [1357]:
print(louvre1[:5])
print(louvre2[:5])
print(louvre3[:5])

['56', '50', '52', '580', '58']
['85', '65', '1583', '65', '75']
['465', '605']


In [1326]:
def getProbaPredecessor(sequences, element, position):
    dic = dict.fromkeys(range(len(tp_k)), 0)
    for seq in sequences:
        key = int(seq[position - 1])
        dic[key] += 1
    res = {key: dic[key] / len(sequences) 
                        for key in dic.keys()}
    values = sum(dic.values())
    res = {key : dic[key] / values for key in dic.keys()}
    return res

In [1327]:
def getProbaSuccessor(sequences, element, position):
    dic = dict.fromkeys(range(len(tp_k)), 0)
    for seq in sequences:
        if(len(seq) > position +1):
            key = int(seq[position + 1])
            dic[key] += 1
    res = {key: dic[key] / len(sequences) 
                        for key in dic.keys()}
    #values = sum(dic.values())
    #res = {key : dic[key] / values for key in dic.keys()}
    return res

In [1328]:
getProbaPredecessor(louvre2, 4, 1)

{0: 0.05,
 1: 0.125,
 2: 0.1,
 3: 0.0,
 4: 0.15,
 5: 0.0,
 6: 0.15,
 7: 0.025,
 8: 0.35,
 9: 0.05}

In [1358]:
getProbaSuccessor(louvre1, 5, 0)

{0: 0.15789473684210525,
 1: 0.02631578947368421,
 2: 0.07894736842105263,
 3: 0.10526315789473684,
 4: 0.07894736842105263,
 5: 0.0,
 6: 0.10526315789473684,
 7: 0.0,
 8: 0.3684210526315789,
 9: 0.07894736842105263}

In [1359]:
getProbaSuccessor(louvre2, 5, 1)

{0: 0.0,
 1: 0.0,
 2: 0.05,
 3: 0.0,
 4: 0.0,
 5: 0.0,
 6: 0.0,
 7: 0.0,
 8: 0.05,
 9: 0.0}

In [1360]:
getProbaSuccessor(louvre3, 5, 2)

{0: 0.0,
 1: 0.0,
 2: 0.0,
 3: 0.0,
 4: 0.0,
 5: 0.0,
 6: 0.0,
 7: 0.0,
 8: 0.0,
 9: 0.0}

## Creation of the FPTA 

In order to be able to make our predictions, we first need to create our prefix tree, representing all the possible sequences read by the tree. 

We start by creating the *TrieNode* class. It represents a node of our tree. It takes as attribute:
- a string of characters 'text', which represents the label of each node.
- an 'is_word' boolean initialized to False. It becomes True if it has no children (i.e. if the word is finished).
- a list of children, to which we initialize for all nodes the first element representing the number of words ending at this node (completes the boolean presented above)
- an integer 'state', which represents the id of the node. It is unique for each node of the tree.
- an 'is_displayed' boolean initialized to False. It is used when displaying the tree, when there are loops so that it is displayed only once.
- a list of precedents, which grows as you go. All the precedents of each node are added to it. For the root node, we consider that its predecessor is itself.

For the *PrefixTree* class, which represents the tree as such, we have:
- a *TrieNode* which represents the root node.
- class functions that we will detail below

### Class functions of the *PrefixTree*.

#### Insert function:
It allows us to insert a word (taken as a parameter), starting from a certain node (current parameter). To do so, we enumerate the word (each character is taken one by one with its corresponding index), then, if the character is not present in the list of children, we will create a node and then increment the indices. If there already exists a child node with the same character, we increment its frequency by 1. When the word is finished, we pass the boolean to True, and we increment the word counter by 1. 

<img src="https://upload.wikimedia.org/wikipedia/commons/thumb/b/be/Trie_example.svg/1200px-Trie_example.svg.png" alt="Drawing" style="width: 200px;"/>

In [1330]:
class TrieNode:
    _COUNTER_NODE = 1
    def __init__(self, text = "λ"):
        self.text = text
        self.is_word = False
        self.children = list()
        self.children.append(['#', 0, self]) #1st element:SYMB 2nd element:FREQUENCY 3rd element:CHILDREN 
        self.state = TrieNode._COUNTER_NODE
        TrieNode._COUNTER_NODE += 1
        self.previous = []

    def get_frequency(self): # Not counting the number of times a word ends 
        freq = 0
        for child in self.children[1:]:
            freq += child[1]
        return freq
    
    def get_frequency2(self):
        freq = 0
        for child in self.children:
            freq += child[1]
        return freq

    def __str__(self):
        return 'symb:{} state:{} -> children:{}'.format(self.text, self.state, self.children)

class PrefixTree:
    def __init__(self):
        self.root = TrieNode()
        self.matrice_transition = pd.DataFrame()
        self.matrice_emission = pd.DataFrame()

    def insert(self, word, current = None):
        '''
        Creates a given word in the trie
        '''
        if not current: 
            current = self.root
            pre = self.root
            
        for i, char in enumerate(word):
            
            if char not in [x[0] for x in current.children]:
                prefix = word[0:i+1]
                current.children.append([char, 1, TrieNode(prefix)])
                
                if pre not in current.previous:
                    current.previous.append(pre)
                    
                pre = current
                current = current.children[-1][2]

            else:
                for child in current.children:
                    if child[0] == char:
                        child[1] += 1
                        
                        if pre not in current.previous:
                            current.previous.append(pre)
                            
                        pre = current
                        current = child[2]

        current.is_word = True   
        current.children[0][1] += 1
        
    def starts_with(self, prefix, current = None):
        '''
        Returns a list of all words beginning with the given prefix, or
        an empty list if no words begin with that prefix.
        '''
        words = list()
        if not current:
            current = self.root
        for char in prefix:
            if char not in [x[0] for x in current.children]:
                return list()
            for child in current.children:
                  if child[0] == char:
                    current = child[2]
        self.__child_words_for(current, words)
        return words
    
    def __child_words_for(self, node, words):
        '''
        Private helper function. Cycles through all children
        of node recursively, adding them to words if they
        constitute whole words (as opposed to merely prefixes).
        '''
        if node.is_word:
            words.append(node.text)
        for child in node.children[1:]:
            self.__child_words_for(child[2], words)
    
    def size(self, current = None, visited = None):
        '''
        Returns the size of this prefix tree, defined
        as the total number of nodes in the tree.
        '''
        # By default, get the size of the whole trie, starting at the root
        if not current:
            current = self.root
        
        if not visited:
            visited = []
            visited.append(current)
            
        count = 1
        
        for child in current.children[1:]:
            if(child[2] not in visited):
                visited.append(child[2])
                count += self.size(child[2], visited)
                
        return count

    def leaf(self, current = None, visited = None):
        if not current: 
            current = self.root
            
        if not visited:
            visited = []
            
        count = 0
        tmp = 0
        
        self.__child_words_for(current, count)        
        return count
    
    def __leaves_for(self, current, count, visited):
        if current.is_word == True:
            count += 1
        for child in current.children[1:]:
            if(child[2] not in visited):
                visited.append(child[2])
                self.__leaves_for(child[2], count, visited)
    
    def transform2proba(self, current = None):
        if not current:
            current = self.root
            
        freq = current.get_frequency2()
        for child in current.children:
            if(isinstance(child[1], int) == False):
                return 0
            else: 
                child[1] = child[1]/freq
                if child[2].state != current.state:
                    current = child[2]    
                    self.transform2proba(current)
    
    def group_transition(self, node):
        result = []
        for child in node.children:
            list_child = [x for x in node.children if x[2].state == child[2].state]
            if list_child not in result:
                result.append(list_child)

        return result

    def transform_to_HMMT(self, current = None, visited = None):
        if not current:
            current = self.root

        if not visited:
            visited = []
            visited.append(current)

        freq = current.get_frequency2()

        for child in current.children:
            if(len(child) <= 3):
                trans = self.group_transition(current)

                for sub in trans:
                    sum_ = 0
                    for node in sub:
                        sum_ += node[1]
                    for node in sub:
                        node[1] /= sum_
                        node.append(sum_)              

        for child in current.children[1:]:
            if child[2] not in visited:  
                visited.append(current) 
                self.transform_to_HMMT(child[2], visited)   
                
    def create_matrix_transition(self, current = None, visited = None):
        if not current:
            current = self.root
        
        if not visited:
            visited = []   
            
        for i in range(len(current.children)):
            listt = self.group_transition(current)
            for c in listt:
                self.matrice_transition.loc[current.text, c[0][2].text] = round(c[0][3], 2)
            visited.append(current)
                
        for i in range(len(current.children)):
            if(current.children[i][2] not in visited):
                self.create_matrix_transition(current.children[i][2], visited) 
                
        self.matrice_transition = self.matrice_transition.fillna(0).astype('float')
        #self.matrice_transition = self.matrice_transition.to_numpy(dtype = 'float')
        
    def create_matrix_emission(self, current = None, visited = None):
        if not current:
            current = self.root
        
        if not visited:
            visited = []
            
        for i in range(len(current.children)):
            self.matrice_emission.loc[current.text, current.children[i][0]] = round(current.children[i][1], 2)
            visited.append(current)
        
        for i in range(len(current.children)):
            if(current.children[i][2] not in visited):
                self.create_matrix_emission(current.children[i][2], visited) 
        
                
        self.matrice_emission = self.matrice_emission.fillna(0).astype('float')
        self.matrice_emission = self.matrice_emission.reindex(sorted(self.matrice_emission.columns), axis=1)
        #self.matrice_emission = self.matrice_emission.to_numpy(dtype = 'float')
        
    def viterbi_path(self, observations, prior = None, scaled=True, ret_loglik=False):
        '''Finds the most-probable (Viterbi) path through the HMM state trellis
        Notation:
            Z[t] := Observation at time t
            Q[t] := Hidden state at time t
        Inputs:
            prior: np.array(num_hid)
                prior[i] := Pr(Q[0] == i)
            transmat: np.ndarray((num_hid,num_hid))
                transmat[i,j] := Pr(Q[t+1] == j | Q[t] == i)
            obslik: np.ndarray((num_hid,num_obs))
                obslik[i,t] := Pr(Z[t] | Q[t] == i)
            scaled: bool
                whether or not to normalize the probability trellis along the way
                doing so prevents underflow by repeated multiplications of probabilities
            ret_loglik: bool
                whether or not to return the log-likelihood of the best path
        Outputs:
            path: np.array(num_obs)
                path[t] := Q[t]
        '''
        obslik = np.array([self.matrice_emission.to_numpy()[:, z] for z in observations]).T
        num_hid = obslik.shape[0] # number of hidden states
        num_obs = obslik.shape[1] # number of observations (not observation *states*)
        
        if not prior:
            prior = np.zeros(self.matrice_transition.to_numpy().shape[0])
            prior[0] = 1

        # trellis_prob[i,t] := Pr((best sequence of length t-1 goes to state i), Z[1:(t+1)])
        trellis_prob = np.zeros((num_hid,num_obs))

        # trellis_state[i,t] := best predecessor state given that we ended up in state i at t
        trellis_state = np.zeros((num_hid,num_obs), dtype=int) # int because its elements will be used as indicies

        path = np.zeros(num_obs, dtype=int) # int because its elements will be used as indicies

        trellis_prob[:,0] = np.dot(prior, obslik[:,0]) # element-wise mult

        if scaled:
            scale = np.ones(num_obs) # only instantiated if necessary to save memory
            scale[0] = 1.0 / np.sum(trellis_prob[:,0])
            trellis_prob[:,0] *= scale[0]

        trellis_state[:,0] = 0 # arbitrary value since t == 0 has no predecessor

        for t in range(1, num_obs):
            for j in range(num_hid):
                trans_probs = trellis_prob[:, t - 1] * self.matrice_transition.to_numpy()[:, j] # element-wise mult
                trellis_state[j,t] = trans_probs.argmax()
                trellis_prob[j,t] = trans_probs[trellis_state[j,t]] # max of trans_probs
                trellis_prob[j,t] *= obslik[j,t]
            if scaled:
                scale[t] = 1.0 / np.sum(trellis_prob[:,t])
                trellis_prob[:,t] *= scale[t]

        path[-1] = trellis_prob[:,-1].argmax()
        for t in range(num_obs-2, -1, -1):
            path[t] = trellis_state[(path[t+1]), t+1]

        if not ret_loglik:
            return path
        else:
            if scaled:
                loglik = -np.sum(np.log(scale))
            else:
                p = trellis_prob[path[-1], -1]
                loglik = p
            return path, loglik
    
    def find_node(self, text, current = None, visited = None):
        if not current:
            current = self.root
            
        if not visited:
            visited = []
        
        if current.text == text:
            return current
        
        for child in current.children[1:]:
            if(child[2] not in visited):
                current = child[2]
                visited.append(current)
                found = self.find_node(text, current = current, visited = visited)
                if found: 
                    return found
    
    def prediction(self, seq, L):
        S = dict()
        path = self.viterbi_path(seq)
        current = self.find_node(text = self.matrice_transition.index[path[-1] - 1])
        return self.suffixes(current, 1, S, '', L)
    
    def suffixes(self, current, probability, S, suffix, L):
        if L == 0:
            return S
        
        else:
            for child in current.children[1:]:
                suffix += child[0]
                probability = probability * child[1] * child[3]
                S[suffix] = probability
                found = self.suffixes(child[2], probability, S, suffix, L-1)
                if found:
                    return found
                          
    def forward(self, V, a, b, initial_distribution):
        alpha = np.zeros((V.shape[0], a.shape[0]))
        alpha[0, :] = initial_distribution * b[:, V[0] + 1]
        #print(alpha)
        for t in range(1, V.shape[0]):
            for j in range(a.shape[0]):
                # Matrix Computation Steps
                #                  ((1x2) . (1x2))      *     (1)
                #                        (1)            *     (1)
                alpha[t, j] = alpha[t - 1].dot(a[:, j]) * b[j, V[t] + 1]
            
        return alpha
 
    def backward(self, V, a, b):
        beta = np.zeros((V.shape[0], a.shape[0]))

        # setting beta(T) = 1
        beta[V.shape[0] - 1] = np.ones((a.shape[0]))

        # Loop in backward way from T-1 to
        # Due to python indexing the actual loop will be T-2 to 0
        for t in range(V.shape[0] - 2, -1, -1):
            for j in range(a.shape[0]):
                beta[t, j] = (beta[t + 1] * b[:, V[t + 1] + 1]).dot(a[j, :])
        
        return beta
 
    def baum_welch(self, obs, initial_distribution = None, n_iter = 100):
        
        a = self.matrice_transition.to_numpy()
        b = self.matrice_emission.to_numpy()
        
        
        if not initial_distribution:
            initial_distribution = np.full(self.matrice_transition.to_numpy().shape[0], 1/self.matrice_transition.to_numpy().shape[0])
            #initial_distribution[0] = 1
        
        M = a.shape[0]
        T = len(obs)

        for n in range(n_iter):
            alpha = self.forward(obs, a, b, initial_distribution)
            beta = self.backward(obs, a, b)

            xi = np.zeros((M, M, T - 1))
            for t in range(T - 1):
                denominator = np.dot(np.dot(alpha[t, :].T, a) * b[:, obs[t + 1] + 1].T, beta[t + 1, :])
                for i in range(M):
                    numerator = alpha[t, i] * a[i, :] * b[:, obs[t + 1] + 1].T * beta[t + 1, :].T
                    xi[i, :, t] = numerator / denominator

            gamma = np.sum(xi, axis=1) 
            a = np.sum(xi, 2) / np.sum(gamma, axis=1).reshape((-1, 1))

            # Add additional T'th element in gamma
            gamma = np.hstack((gamma, np.sum(xi[:, :, T - 2], axis=0).reshape((-1, 1))))

            K = b.shape[1]
            denominator = np.sum(gamma, axis=1)
            for l in range(K):
                b[:, l] = np.sum(gamma[:, obs == l + 1], axis=1)

            b = np.divide(b, denominator.reshape((-1, 1)))
            
        tmp1 = self.matrice_transition
        tmp2 = self.matrice_emission
        
        self.matrice_transition = pd.DataFrame(a, index = tmp1.index, columns = tmp1.columns)
        self.matrice_emission = pd.DataFrame(b, index = tmp2.index, columns = tmp2.columns)
    
    def display(self):
        '''
        Prints the contents of this prefix tree.
        '''
        print('====================================================================================')
        self.__displayHelper(self.root)
        print('====================================================================================\n')
    
    def __displayHelper(self, current, visited = None):
        '''
        Private helper for printing the contents of this prefix tree.
        '''
        if not visited: 
            visited = []
            visited.append(current)
        
        print(current, "\n")
        
        for child in current.children[1:]:
            if child[2] not in visited:
                visited.append(current)
                self.__displayHelper(child[2], visited)

In [1331]:
seq = ['aaa', 'aaba', 'ababa', 'bb', 'cb', 'bbaaa']
seq.sort()
print(seq)

['aaa', 'aaba', 'ababa', 'bb', 'bbaaa', 'cb']


In [1332]:
trie = PrefixTree()
for seqq in seq:
    trie.insert(seqq)

## Creation of the DFA with the FPTA:

To transform a prefix tree into a frequency automaton (and thus probabilistic), a state-merging algorithm is used. Currently, the best one has been created by Colin de la Higuera. It is implemented from scratch below:

First of all, we have in parameters the sequences and an alpha parameter [0, 1]. 
We create the tree then we initialize 2 lists: red & blue: 
- Red stores all nodes that have already been defined as representative nodes and will be included in the final output model.
- Blue stores nodes that have not yet been tested.

The purpose of the algorithm is to keep only the red states. At the beginning,RED contains only the root node, whileBlue contains the immediate successor nodes of the initial node: the child nodes of the root. When executing the external loop of the algorithm, the first qb node in BLUE is chosen. If there is a qr node in RED compatible with qb, then qb and its successor nodes are merged with qr and the corresponding successor nodes of qr, respectively, as described in the Merge&Fold part. If qb is not compatible with any of the states of RED, it will be promoted, i.e. it will switch to RED, and its respective children will switch to BLUE.

#### Merge&Fold operation:
Given the DFA, the merge operation takes 2 states q and q′, where q is a RED state and q′ is a BLUE state compatible with the Hoeffding test. Merge operation consists of 2 steps. First, if q′est is a final state, then q becomes one as well (if it was not already the case) and the number of sequences ending in q′ is added to the number of sequences ending in q. Second, incoming arcs at q′sont redirected to q. If such arcs already exist, the number of passing sequences in each incoming arc is added to the existing arc. The Fold operation is a recursive function consisting in merging the successor nodes q′ into q and the corresponding successor nodes of q respectively.

In [1333]:
def Original_Alergia(sequences, alpha):
    '''
    Original version of the Alergia Algorithm
    
    Arguments: 
    sequences -- the sequences to build the Trie and then convert it in a PFA
    alpha -- an int between 0 and 1. It represents a parameter of the Hoeffding bounds. 
    
    Return:
    A DPA according to the sequences we have
    '''
    print('Starting building corresponding Trie \n')
    trie = PrefixTree()
    trie.alphabet = list({l for word in sequences for l in word})
    for subseq in sequences:
        trie.insert(subseq)
    print(trie.size())
    #print(trie.leaf())
    #initial = trie.size()
    red = []
    red.append(trie.root)
    blue = []
    for x in trie.root.children[1:]:
        if(x[2] not in blue):
            blue.append(x[2])
    t0 = 0.5
    
    print('Running Alergia on trie \n')
    while(len(blue) > 0):
        qb = blue[0]
        blue.remove(qb)
        promote = True
        if(qb.get_frequency() >= t0):
            for qr in red:
                if(Original_AlergiaCompatible(trie, qr, qb, alpha)):
                    #print('Merge accepted...')
                    #print(qr)
                    #print(qb)
                    trie = Original_MergeFold(trie, qr, qb)
                    promote = False
                    for child in qr.children[1:]:    
                        if(child[2] not in blue and child[2].state != qr.state and child[2] not in red):
                            blue.append(child[2])
                    break
        
            if(promote == True):
                #print('No merge possible...')
                red.append(qb)
                for child in qb.children[1:]:
                    if(child[2] not in blue and child[2] not in red):
                        blue.append(child[2])
        else:
            continue
    
    trie.transform2proba()
    print('Algo done')
    return trie

def Original_AlergiaCompatible(trie, qr, qb, alpha):
    '''
    Sub-method of the Alergia main one. It tells us if 2 nodes (a red and a blue) are compatible
    
    Arguments:
    trie -- the current trie we are working on
    qr -- a red node 
    qb -- a blue node
    alpha -- parameter alpha mentionned above
    
    Return:
    a boolean saying whether or not the 2 nodes in arguments are compatible
    '''
    correct = True
    if(Original_AlergiaTest(qr.children[0][1], qr.get_frequency(), qb.children[0][1], qb.get_frequency(), alpha) == False):
        correct = False
    
    for a in trie.alphabet:
        for children_r in qr.children:
            for children_b in qb.children:
                if(a == children_r[1] and a == children_b[1]):
                    if(Original_AlergiaTest(children_r[2], children_r.get_frequency(), children_b[2], children_b.get_frequency(), alpha) == False):
                        correct = False
    return correct
  
def Original_AlergiaTest(f1, n1, f2, n2, alpha):
    '''
    Sub-method of the Alergia compatible method. It computes the hoeffding bounds and compare it to the gamma threshold
    
    Arguments:
    f1 -- incoming frequency of the node 1
    n1 -- number of times a word ends in node 1
    f2 -- incoming frequency of the node 2
    n2 -- number of times a word ends in node 2
    1 & 2 refers to the nodes we want to compare (i.e a node Red and a node Blue)
    alpha -- same parameter alpha as mentionned
    
    Return:
    True if gamma < hoeffding (then, the nodes are compatible)
    False if not
    '''
    gamma = math.fabs(f1/n1 - f2/n2)
    root = math.sqrt(1/(2*math.log(2/alpha)))
    summ = (1/math.sqrt(n1)) + (1/math.sqrt(n2))
    hoeffding = root * summ
    return gamma < hoeffding

def Original_MergeFold(trie, qr, qb): 
    '''
    MERGE OPERATION: 
    To merge a BLUE node into a RED node, all the ingoing links of BLUE node are redirected to the RED node. 
    If the RED node already has an ingoing link with the same item attribute as a redirected link, then only the frequencies are added. 
    Next, the BLUE node attribute is added to the RED node attribute.
    
    FOLD OPERATION:
    When a BLUE node merges into a RED node, all children of BLUE node are recursively merged into children of the RED node with the same item attribute link.
    If the link with an item attribute doesn't exist in the RED node, a new child is inserted.
    
    Arguments:
    trie -- the current Trie we are working on
    qr -- the red node 
    qb -- the blue node to merge
    '''
    q = qb.previous[0]
    
    if qb.text[-1] not in qr.text:
        qr.text = qr.text+ ', ' + qb.text[-1]
        
    for child in q.children:
        if(child[2].state == qb.state):
            child[2] = qr 
            
    qr.children[0][1] += qb.children[0][1]
    
    words = trie.starts_with('', current = qb)
    
    for word in words:
        trie.insert(word)
        
    return trie

In [1334]:
%time dfa = Original_Alergia(seq2j, 0.5)
print(dfa.size())

Starting building corresponding Trie 

115
Running Alergia on trie 

Algo done
Wall time: 5 ms
11


In [1348]:
dfa.transform_to_HMMT()
dfa.create_matrix_transition()
dfa.create_matrix_emission()

In [1336]:
def Alergia(sequences, alpha):
    '''
    Our version of the Alergia algorithm. It tries to recursively merge all the pairs of nodes, from the root to the leaves.
    Unlike the original version, the difference is that it does not test the compatibility of children of these pairs of nodes.
    
    Arguments: 
    sequences -- the sequences to build the Trie and then convert it in a PFA
    alpha -- an int between 0 and 1. It represents a parameter of the Hoeffding bounds. 
    
    Return:
    A DPA according to the sequences we have
    '''
    
    print('Starting building corresponding Trie \n')
    
    trie = PrefixTree()
    
    trie.alphabet = list({l for word in sequences for l in word})
    
    for subseq in sequences:
        trie.insert(subseq)
        
    #print(trie.size())
    #print(trie.leaf())
    #initial = trie.size()
    red = []
    
    red.append(trie.root)
    
    blue = []
    
    for x in trie.root.children[1:]:
        if(x[2] not in blue):
            blue.append(x[2])
            
    t0 = 0.5
    
    print('Running Alergia on trie \n')
    
    while(len(blue) > 0):
        qb = blue[0]
        blue.remove(qb)
        promote = True
        if(qb.get_frequency() >= t0):
            for qr in red:
                if(AlergiaCompatible(trie, qr, qb, alpha)):
                    #print('Merge accepted...')
                    #print(qr)
                    #print(qb)
                    trie = MergeFold(trie, qr, qb)
                    promote = False
                    for child in qr.children[1:]:    
                        if(child[2] not in blue and child[2].state != qr.state and child[2] not in red):
                            blue.append(child[2])
                    break
        
            if(promote == True):
                #print('No merge possible...')
                red.append(qb)
                for child in qb.children[1:]:
                    if(child[2] not in blue and child[2] not in red):
                        blue.append(child[2])
        else:
            continue
    
    trie.transform2proba()
    print('Algo done')
    return trie

def AlergiaCompatible(trie, qr, qb, alpha):
    '''
    Sub-method of the Alergia main one. It tells us if 2 nodes (a red and a blue) are compatible
    
    Arguments:
    trie -- the current trie we are working on
    qr -- a red node 
    qb -- a blue node
    alpha -- parameter alpha mentionned above
    
    Return:
    a boolean saying whether or not the 2 nodes in arguments are compatible
    '''
    correct = True
    
    if(AlergiaTest(qr.children[0][1], qr.get_frequency(), qb.children[0][1], qb.get_frequency(), alpha) == False):
        correct = False
    
    return correct
  
def AlergiaTest(f1, n1, f2, n2, alpha):
    '''
    Sub-method of the Alergia compatible method. It computes the hoeffding bounds and compare it to the gamma threshold
    
    Arguments:
    f1 -- incoming frequency of the node 1
    n1 -- number of times a word ends in node 1
    f2 -- incoming frequency of the node 2
    n2 -- number of times a word ends in node 2
    1 & 2 refers to the nodes we want to compare (i.e a node Red and a node Blue)
    alpha -- same parameter alpha as mentionned
    
    Return:
    True if gamma < hoeffding (then, the nodes are compatible)
    False if not
    '''
    gamma = math.fabs(f1/n1 - f2/n2)
    root = math.sqrt(1/(2*math.log(2/alpha)))
    summ = (1/math.sqrt(n1)) + (1/math.sqrt(n2))
    hoeffding = root * summ
    return gamma < hoeffding

def MergeFold(trie, qr, qb): 
    '''
    MERGE OPERATION: 
    To merge a BLUE node into a RED node, all the ingoing links of BLUE node are redirected to the RED node. 
    If the RED node already has an ingoing link with the same item attribute as a redirected link, then only the frequencies are added. 
    Next, the BLUE node attribute is added to the RED node attribute.
    
    FOLD OPERATION:
    When a BLUE node merges into a RED node, all children of BLUE node are recursively merged into children of the RED node with the same item attribute link.
    If the link with an item attribute doesn't exist in the RED node, a new child is inserted.
    
    Arguments:
    trie -- the current Trie we are working on
    qr -- the red node 
    qb -- the blue node to merge
    '''
    q = qb.previous[0]
    
    if qb.text[-1] not in qr.text:
        qr.text = qr.text+ ', ' + qb.text[-1]
        
    for child in q.children:
        if(child[2].state == qb.state):
            child[2] = qr 
            
    qr.children[0][1] += qb.children[0][1]
    
    words = trie.starts_with('', current = qb)
    
    for word in words:
        trie.insert(word)
        
    return trie

In [1349]:
%time dfa_2 = Alergia(seq2j, 1/125)
dfa_2.size()

Starting building corresponding Trie 

Running Alergia on trie 

Algo done
Wall time: 3.98 ms


29

In [1353]:
dfa_2.transform_to_HMMT()
dfa_2.create_matrix_transition()
dfa_2.create_matrix_emission()

In [1354]:
dfa_2.display()

symb:λ, 9, 1, 7, 8, 3, 0 state:256 -> children:[['#', 0.07738095238095238, <__main__.TrieNode object at 0x0000023F906C1988>, 0.6086956521739131], ['9', 0.0744047619047619, <__main__.TrieNode object at 0x0000023F906C1988>, 0.6086956521739131], ['1', 0.11309523809523808, <__main__.TrieNode object at 0x0000023F906C1988>, 0.6086956521739131], ['7', 0.05654761904761904, <__main__.TrieNode object at 0x0000023F906C1988>, 0.6086956521739131], ['8', 0.33035714285714285, <__main__.TrieNode object at 0x0000023F906C1988>, 0.6086956521739131], ['3', 0.10416666666666667, <__main__.TrieNode object at 0x0000023F906C1988>, 0.6086956521739131], ['0', 0.244047619047619, <__main__.TrieNode object at 0x0000023F906C1988>, 0.6086956521739131], ['2', 0.24550898203592814, <__main__.TrieNode object at 0x0000023F906CEE08>, 0.302536231884058], ['4', 0.46107784431137727, <__main__.TrieNode object at 0x0000023F906CEE08>, 0.302536231884058], ['6', 0.2934131736526946, <__main__.TrieNode object at 0x0000023F906CEE08>,

# Predictions

In [1351]:
s = dfa_2.prediction(np.array([4, 4, 2, 5]), 1)
print(s)

{'9': 0.04528985507246377}


In [1352]:
s = dfa.prediction(np.array([4, 4, 2, 5]), 1)
print(s)

{'9': 0.0444104134762634}
